# All detailed results

This notebooks holds all detailled results that are mentioned in the text. Re-execute to update values.

In [1]:
import functools

import numpy as np
import xarray as xr

TOTAL_EUROPEAN_LAND_MASS_KM2 = 4920000
THRESHOLDS = [0.005, 0.01, 0.015, 0.02, 0.03]
TECHS = ["roof", "util", "offshore"]
ALL_TECHS = TECHS + ["wind"]


def calculate_data(path_to_xy_data):
    xy = xr.open_dataset(path_to_xy_data)
    data = (
        xr
        .ones_like(xy.cost.sum("scenario"))
        .expand_dims(tech=TECHS, threshold=THRESHOLDS)
    ) * np.nan

    cost_optimal_data = xy.isel(scenario=xy.cost.argmin("scenario"))

    for tech in TECHS:
        conditions = [
            xy[other_tech] <= cost_optimal_data[other_tech]
            for other_tech in ALL_TECHS
            if other_tech != tech
        ]
        tech_mask = functools.reduce(lambda x, y: x & y, conditions)
        for threshold in THRESHOLDS:
            absolute_threshold = threshold * TOTAL_EUROPEAN_LAND_MASS_KM2
            mask = tech_mask & (xy.land_use <= absolute_threshold)
            index_of_new_cost_optimal_scenarios = xy.where(mask).cost.argmin("scenario")
            cost = xy.isel(scenario=index_of_new_cost_optimal_scenarios).cost
            data.loc[{"tech": tech, "threshold": threshold}] = (cost - cost_optimal_data.cost) / cost_optimal_data.cost
    return data

## Expected results

In [2]:
xy = xr.open_dataset("../build/euler/output/national/land-use/xy.nc")

In [3]:
expected = xy.mean("sample_id")

In [4]:
cost_optimal_expected_scenario = expected.to_dataframe().cost.idxmin()

In [5]:
total = expected.sel(scenario=cost_optimal_expected_scenario).land_use.item()
relative = total / TOTAL_EUROPEAN_LAND_MASS_KM2 * 100
print(f"Expected land use of cost-optimal system design using expected cost:")
print(f"{total:.0f} km2 ({relative:.1f}%)")

Expected land use of cost-optimal system design using expected cost:
91615 km2 (1.9%)


## Land limits

In [6]:
penalties = calculate_data("../build/euler/output/national/land-use/xy.nc")

In [7]:
for threshold in THRESHOLDS:
    filtered_penalities = penalties.sel(threshold=threshold, tech="util")
    share_penalty = ((filtered_penalities > 0).sum() / filtered_penalities.count()).item() * 100
    print(f"Limit of {threshold * 100:.1f}% using any tech requires penalities in {share_penalty:.0f}% of the time.")

Limit of 0.5% using any tech requires penalities in 79% of the time.
Limit of 1.0% using any tech requires penalities in 74% of the time.
Limit of 1.5% using any tech requires penalities in 59% of the time.
Limit of 2.0% using any tech requires penalities in 31% of the time.
Limit of 3.0% using any tech requires penalities in 2% of the time.
